<a href="https://colab.research.google.com/github/minml67948/half_life_weighted/blob/main/half_life_covtype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# まとめ




In [ ]:
"""

model  :  rmse       :  time
----------------------------
  hlw  :  0.5803432  :  2h
  lgbm :  0.8311869  :  12s
  xgb  :  0.72971433 :  12s
  rf   :  0.6519643  :  44s

next
・すべてのモデルを分類問題用に変更
・hlwの実行時間の改善

"""

# 半減期加重の実装（回帰）

In [ ]:
import jax.numpy as jnp
from jax import jit as jjit
from functools import partial
from jax import vmap
from jax.lax import cond, while_loop

############自作モデルの実装##########

#誤差を求める関数
@partial(jjit)
def rmse(y, p):
    return jnp.sqrt(((y - p)**2).mean())

#予測する関数(１行)
@partial(jjit)
def predict(row, model_x, model_y, T,replace_value):
    t = jnp.abs(model_x - row)
    w = (1/2)**(t / T)
    w = jnp.prod(w, axis=1)
    p = jnp.average(model_y, weights=w)
    p = jnp.nan_to_num(p, replace_value)
    return p

#予測する関数（全行）
@partial(jjit)
def predict_array(x, model_x, model_y, T,replace_value):
    return vmap(lambda row : predict(row, model_x, model_y, T, replace_value))(x)

#学習関数modelとtunerを引数で設定する必要がある（ランダムな要素を排除）
@partial(jjit)
def fit(T, model_x, model_y, tuner_x, tuner_y, replace_value, rate = 0.7):
    #初期状態の予測値・誤差・半減期
    same_cnt = 0 #精度が改善できなかった連続回数
    tuner_p = predict_array(tuner_x, model_x, model_y, T, replace_value)
    err = rmse(tuner_y, tuner_p)
    i = 0
    row,col = model_x.shape
    #半減期と学習率の更新
    def update_loop(params):
        i, err, T, same_cnt = params
        err_ex = err
        #半減期を縮小
        T_ = T.at[i%col].set(T[i%col] * rate)
        tuner_p = predict_array(tuner_x, model_x, model_y, T_, replace_value)
        err_ = rmse(tuner_y, tuner_p)
        err, T = cond(err_ < err, lambda: [err_, T_], lambda: [err, T])
        #半減期を拡大
        T_ = T.at[i%col].set(T[i%col] / rate)
        tuner_p = predict_array(tuner_x, model_x, model_y, T_, replace_value)
        err_ = rmse(tuner_y, tuner_p)
        err, T  = cond(err_ < err, lambda: [err_, T_], lambda: [err, T])
        #連続で改善に失敗した回数を更新
        same_cnt =  cond(err_ex == err, lambda: same_cnt+1, lambda: 0)
        i+=1
        params = [i, err, T, same_cnt]
        return params
    params = [i, err, T, same_cnt]
    params = while_loop(lambda params: params[3] < col, update_loop, params)
    [i, err, T, same_cnt] = params
    return {"T": T, "err": err, "try_cnt": i+1}

#カリフォルニアデータの学習と予測

In [23]:

#カリフォルニア住宅価格データをロード
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_covtype

covtype = fetch_covtype()

#訓練データと検証データに分割
x = covtype.data
y = covtype.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, train_size = 0.1)



In [27]:
X_train.shape

(58101, 54)

#half life weighted

In [32]:
#half_lifeのパラメータ設定
model_x, tuner_x, model_y, tuner_y = train_test_split(X_train, y_train, test_size = 0.5, train_size=0.5)
T_first = (jnp.max(model_x, axis=0) - jnp.min(model_x, axis=0))/2
replace_value = jnp.mean(model_y)

#学習
result = fit( T_first, model_x, model_y, tuner_x, tuner_y, replace_value)
T = result["T"]
print(result)

#予測
p_test = predict_array(X_test,model_x, model_y, T, replace_value)

#精度
print("精度")
print(rmse(y_test,p_test))

精度
0.5803432


#lightGBM

In [33]:
#lightGBM
import lightgbm as lgb
#学習
model = lgb.LGBMRegressor(
    random_state = 42,
)

model.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test), (X_train, y_train)],
    verbose=-1 # 学習ログを省略
)

#予測
p_lgb = model.predict(X_test)

#精度
print("精度")
print(rmse(y_test,p_lgb))
     

/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


精度
0.8311869


#xgboost

In [34]:
#XGboost
import xgboost as xgb
#学習
xgbX_train, xgbX_test, xgby_train, xgby_test = train_test_split(X_train, y_train, test_size = 0.2)
dtrain = xgb.DMatrix(xgbX_train, label=xgby_train)
dvalid = xgb.DMatrix(xgbX_test, label=xgby_test)

params = {
        'objective': 'reg:squarederror','silent':1, 'random_state':1234, 
        # 学習用の指標 (RMSE)
        'eval_metric': 'rmse',
    }
num_round = 500
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]#訓練データはdtrain、評価用のテストデータはdvalidと設定
model = xgb.train(
    params,
    dtrain,#訓練データ
    num_round,#設定した学習回数
    early_stopping_rounds=20,
    evals=watchlist,
    verbose_eval=0
)

#予測
dtest = xgb.DMatrix(X_test)
p_xgb = model.predict(dtest, ntree_limit = model.best_ntree_limit)

#精度
print("精度")
print(rmse(y_test,p_xgb))

[11:54:27] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


精度
0.72971433


#random forest

In [35]:
from sklearn.ensemble import RandomForestRegressor
#学習
clf = RandomForestRegressor(random_state=0)
clf.fit(X_train, y_train)

#予測
p_rf = clf.predict(X_test)

#精度
print("精度")
print(rmse(y_test,p_rf))

精度
0.6519643


#lightBGM ハイパーパラメータチューニング


sklearn

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
import numpy as np


# モデルの定義
gbm = lgb.LGBMRegressor(objective='regression')

# ハイパーパラメータの範囲の指定
param_dist = {
    'learning_rate': np.arange(0.01, 0.1, 0.01),
    'n_estimators': sp_randint(50, 200),
    'max_depth': sp_randint(3, 15),
    'num_leaves': sp_randint(10, 50),
    'min_child_samples': sp_randint(5, 30),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'colsample_bytree': np.arange(0.5, 1.0, 0.1),
    'reg_alpha': np.arange(0, 1, 0.1),
    'reg_lambda': np.arange(0, 1, 0.1),
}

# ランダムサーチによるハイパーパラメータのチューニング
n_iter_search = 20
random_search = RandomizedSearchCV(
    gbm, param_distributions=param_dist, n_iter=n_iter_search, cv=5, scoring='neg_mean_squared_error')
random_search.fit(X_train, y_train)

# 結果の表示
print(random_search.best_params_)
print(random_search.best_score_)

params = random_search.best_params_
model = lgb.LGBMRegressor(objective='regression', **params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse(y_pred,y_test)

{'colsample_bytree': 0.7999999999999999, 'learning_rate': 0.09, 'max_depth': 6, 'min_child_samples': 28, 'n_estimators': 154, 'num_leaves': 48, 'reg_alpha': 0.2, 'reg_lambda': 0.2, 'subsample': 0.7999999999999999}
-0.21426803364911354


Array(0.45894927, dtype=float32)

oputuna


In [ ]:
!pip install optuna

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import mean_squared_error


X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# 目的関数の定義
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000)
    }
    
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train1, y_train1, eval_set=[(X_train2, y_train2)], early_stopping_rounds=50, verbose=False)
    
    y_pred = model.predict(X_train2)
    mse = mean_squared_error(y_train2, y_pred)
    rmse = mse ** 0.5
    
    return rmse

# 最適化の実行
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# 最適なハイパーパラメータでモデルをトレーニング
best_params = study.best_params
model = lgb.LGBMRegressor(**best_params)
model.fit(X_train1, y_train1, eval_set=[(X_train2, y_train2)], early_stopping_rounds=50, verbose=False)

# テストデータの予測
y_pred = model.predict(X_test)

# 結果の出力
print('Best RMSE:', study.best_value)
print('Best Parameters:', study.best_params)
print('Test RMSE:', mean_squared_error(y_test, y_pred, squared=False))


In [ ]:
rmse(y_test, y_pred)

Array(0.44042936, dtype=float32)